<a id="simulating-the-collectors-vault"></a>
# simulating the collector's vault 

The Marvel Future Fight Collector's Vault is the subject of plenty of advice and
warnings (like [this one](https://www.reddit.com/r/future_fight/comments/ch8719/this_is_what_25200000_gold_gets_you_in_the/)
and [this one](https://www.reddit.com/r/future_fight/comments/b0fwtc/collectors_vault_winnings/)). Whether it's a reasonable way to spend gold (if you've got plenty) or just a way to waste it is up for debate. However, an event that you can play at most two times on one day every few weeks is difficult to evaluate by just discussing in the forums. The mechanisms of the event make theoretical statistical considerations [fairly complicated](#why-we-simulate). So, let's simulate it and see what we get.

(For those not at all interested in "how" or "why" and who just want an answer: [should you play Collector's Vault?](#should-you-play-collectors-vault))

## vault types

There have been different variations on the Collector's Vault; the current
incarnation as of this writing is described at 
[the announcement for the July 2022 Collector's Vault](https://forum.netmarble.com/futurefight_en/view/76/1762161), 
and it's been in this format since at least January 2022. There is a "Personal
Vault" where you're *almost* guaranteed to get a 4\* artifact, and there are
"General Vaults" where you and up to 1,500 other players each try to get a
single C.T.P of Veteran or 6\* artifact. This latter kind is the most like older
versions of the Collector's Vault; the major changes in the latest version is a
somewhat lower cost per turn, more opportunities throughout the day to choose a
Vault to "open", and fewer players allowed in each Vault. It's also the first
version in which Netmarble has made the mechanism and probabilities explict,
allowing us to program a simulation.

The General Vaults are of two types:

### general vault - c.t.p. of veteran

| Item | Purchase Limit |Acquisition Chance |
|------|----------------|-------------------|
| Norn Stone of each type x15 | 110000 | 23.6280% |
| Rank 1 Black Anti-Matter x5 | 110000 | 15.7520% |
| Norn Stone of Chaos x5 | 110000 | 15.7520% |
| Gear Up Kit x25 | 110000 | 11.8140% |
| Dimension Debris x30 | 110000 | 11.8140% |
| 3* Type Enhancement Kit x3 | 110000 | 14.6996% |
| 3* Enchanted Uru x1 | 110000 | 6.2999% |
| Tier-2 Mega Advancement Ticket x1 | 5 | 0.0005% |
| Titan Component Pack x40 | No limit | 0.1248% |
| Essence of Dimension x50 | No limit | 0.0720% |
| Cosmic Cube Fragment x60 | No limit | 0.0360% |
| Extreme Obelisk x1 | No limit | 0.0072% |

### general vault - 6* exclusive passive skill artifact
| Item | Purchase Limit |Acquisition Chance |
|------|----------------|-------------------|
| Gear Up Kit x20 | 110000 | 23.6280% |
| Dimension Debris x20 | 110000 | 15.7520% |
| Norn Stone of each type x15 | 110000 | 15.7520% |
| Rank 1 Black Anti-Matter x5 | 110000 | 11.8140% |
| Norn Stone of Chaos x5 | 110000 | 11.8140% |
| 3* ISO-8 x1 | 110000 | 14.6996% |
| Lv. 2 Artifact x1 | 110000 | 6.2999% |
| Mega Uniform Upgrade Ticket: Mythic x1 | 5 | 0.0005% |
| Essence of Dimension x40 | No limit | 0.1248% |
| Cosmic Cube Fragement x50 | No limit | 0.0720% |
| Titan Component Pack x60 | No limit | 0.0360% |
| 6* Rank Up Ticket x1 | No limit | 0.0072% |

Even though the prizes each turn are different, the stats are the same; for instance, the second entry in the list has a 15.7520% chance of being received, whether it's 5 Rank 1 Black Anti-Matter or 20 Dimension Debris. This means simulating the two can be done the same way, and the only change is the specific prizes you might get in the end. We'll come back to this in [making decisions](#making-decisions).



<a id="how-the-vault-and-the-simulation-works"></a>
## how the vault (and the simulation) works

### the assumptions

For our simulation, we must make a few assumptions. First, we assume that people are all taking about the same number of turns, that is, that they aren't acquiring items more quickly or more slowly than anyone else. This isn't true even if they're using the "auto purchase" method (which, for that matter, depends on the connection to Netmarble's servers). If someone is going more slowly than you, however, they'll spend less but are necessarily also less likely to win the big prize---so you're at your worst luck when everyone is going as fast as you are. We're trying to set realistic expectations, and don't want to be unnecessarily "hopeful", so we'll use this "worst case" for our simulation. For the same reason, we're assuming everyone started right away when the Vault opened and that it maxed out the number of users. Finally we assume that at some point the Vault closes. Although it's not realistic, we allow up to 3600*100 turns, or the equivalent of 100 turns per second for the hour that the Vault is open.

### coding the simulation

Python isn't the fastest, but it's pretty easy to read and throw together without being too clever. Let's build up the simulation code while talking about the mechanism of the Vault.

First, we'll note the number of users allowed in each Vault and how much gold you need for each "turn", that is, each time you want to buy an item.

In [1]:
users = 1500
price = 25000

As we noted, we can use either of the General Vaults to check the statistics and run the simulation. We'll store the details for each of them and can decide [later](#making-decisions) if one's better than the other.

In [2]:
inf = float('inf') # this is just a shortcut for those items that have no limit

# item description, number available, probability
ctp_items = [
    ("Norn Stone of each type x15", 110000, 0.236280),
    ("Rank 1 Black Anti-Matter x5", 110000, 0.157520),
    ("Norn Stone of Chaos x5", 110000, .157520),
    ("Gear Up Kit x25", 110000,.118140),
    ("Dimension Debris x30", 110000, .118140),
    ("3* Type Enhancement Kit x3", 110000, .146996),
    ("3* Enchanted Uru x1", 110000, .062999),
    ("Tier-2 Mega Advancement Ticket x1", 5, 0.000005),
    ("Titan Component Pack x40", inf, 0.001248 ),
    ("Essence of Dimension x50", inf, 0.000720),
    ("Cosmic Cube Fragment x60", inf, 0.000360),
    ("Extreme Obelisk x1",inf, 0.000072)
]

artifact_items = [
    ("Gear Up Kit x20", 110000, 0.236280),
    ("Dimension Debris x20", 110000, 0.157520),
    ("Norn Stone of each type x15", 110000, .157520),
    ("Rank 1 Black Anti-Matter x5", 110000,.118140),
    ("Norn Stone of Chaos x5", 110000, .118140),
    ("3* ISO-8 x1", 110000, .146996),
    ("Lv. 2 Artifact x1", 110000, .062999),
    ("Mega Uniform Upgrade Ticket: Mythic x1", 5, 0.000005),
    ("Essence of Dimension x40", inf, 0.001248 ),
    ("Cosmic Cube Fragment x50", inf, 0.000720),
    ("Titan Component Pack x60", inf, 0.000360),
    ("6* Rank Up Ticket x1",inf, 0.000072)
]


Each time you take a "turn" to acquire an item, one is randomly selected from the list. As the items with limited quantities run out, the probabilities of getting the other items changes a bit. For instance, take a look at this recent run, with the "Chance Info" displayed:

![recalculated-probabilities-1](./img/recalculated-probabilities-1.png "A screenshot showing recalculated probabilities after something sells out")
![recalculated-probabilities-2](./img/recalculated-probabilities-1.png "A screenshot showing more recalculated probabilities")
![recalculated-probabilities-3](./img/recalculated-probabilities-1.png "A screenshot showing even more recalculated probabilites")

We'll check for the same thing each time we acquire an item:

In [3]:
import random

def get_item(round_items):
    roll = random.random()
    sum_p = 0.0
    for i in range(len(round_items)):
        (name,num,p) = round_items[i]
        if roll < sum_p + p:
            if num == 1:
                round_items[i] = (name,0,0)
                recalc_p(round_items,p)
            else:
                round_items[i] = (name,num-1,p)
            return i
        else:
            sum_p += p

def recalc_p(round_items,p):
    remaining_p = 1.0-p
    correction = 1 / remaining_p
    for i in range(len(round_items)):
        (name,num,p) = round_items[i]
        round_items[i] = (name,num,p*correction)

As soon as one of the users gets all four of the "memento" items, the Vault closes and nobody can get any more items. Not coincidentally, the "memento" items are those that don't have a limited quantity.

In [4]:
needed_items = []
for i in range(len(ctp_items)):
    if ctp_items[i][1] == inf:
        needed_items.append(i)

def has_needed_items( items ):
    for item in needed_items:
        if item not in items:
            return False
    return True

We also need to keep track of what items the different users have so that we can determine when someone has all the "memento" items (and so that we can tell what items we ended up with). We'll refer to users by number (from 0 to 1499), with "our" user as number 0. If it's our turn, we'll keep track of whatever items we have; if it's a different user's turn we'll just keep track if it's one of the memento items. When we're adding this new item to the list of acquired items, we'll take the opportunity at the same time to see if the user's gathered all the mementos.

In [5]:
user_nums = []
for i in range(users):
    user_nums.append(i)

def add_user_item(user, item, users_items):
    if item in needed_items or user == 0:
        users_items[user].append(item)
        if item in needed_items:
            if has_needed_items(users_items[user]):
                return True
    return False

Finally, we set up the simulated "round" of the Vault. We start with an empty list of items acquired for each user at the beginning of every vault, and since we are editing the probabilities in the table of items we do that work on a copy. Then on every "turn", we shuffle the order in which all the users hit their buttons; we can only calculate one at a time, but we don't want to give the users first in line an advantage. Then each user gets their item and it's added to their acquired items list as above. You'll recall that also checks to see if they've won; if so, our round is over and we can save who the winner was, how many turns we took, and what items we received.

In [6]:
def do_round():
    winner = -1
    turns = -1
    users_items = [[] for i in range(users)]
    round_items = ctp_items.copy()
    for turn in range(3600*100):
        random.shuffle(user_nums)
        for user_num in user_nums:
            if add_user_item( user_num, get_item(round_items), users_items):
                turns = len(users_items[0])
                winner = user_num
                break
        if turns >= 0:
            break
    return (winner,turns,users_items[0])

### running the simulation

Next, we just have to decide how many times we'll run the simulation and what we'll do with the output. Here, we've set it to a modest 10 simulations and we just print the output, which is in the format of a comma-delimited row: the number of the winner, the number of turns our user took, and a list of the indices of all the items our user acquired.

In [7]:
sims = 10 # feel free to increase; depending on your system, each will run in around 0.5-1.5 seconds

for i in range(sims):
    round = do_round()
    print( round[0],round[1],'"' + ','.join(map(str,round[2])) + '"',sep="," )

1065,472,"2,0,0,2,0,4,4,0,4,0,2,6,5,0,0,2,1,1,1,0,0,0,6,5,0,5,2,0,3,0,5,5,3,2,5,0,0,3,1,2,0,1,0,1,3,5,0,0,0,4,0,4,1,6,2,0,1,2,5,3,4,0,5,3,0,4,0,5,3,0,1,4,0,1,0,5,0,6,0,0,3,0,0,3,4,3,4,1,1,6,5,0,0,0,1,4,0,1,1,0,1,0,3,4,4,2,1,4,5,1,5,5,3,2,9,4,0,5,5,0,1,0,0,3,0,0,3,2,3,4,1,0,6,5,5,0,2,1,0,1,3,3,0,4,3,4,0,4,0,0,6,1,4,6,5,0,1,2,0,5,0,6,0,5,5,5,0,0,1,0,5,5,2,2,0,0,1,0,4,0,3,3,3,1,1,5,2,5,3,6,2,3,5,0,2,6,4,2,3,0,1,5,2,0,0,2,0,2,2,0,1,2,6,1,3,1,1,1,4,0,4,0,3,5,5,1,6,5,5,5,6,5,3,1,2,2,4,1,4,3,2,9,5,3,5,4,2,5,0,5,3,0,1,1,4,2,1,0,3,5,5,4,1,0,5,6,6,0,6,5,1,2,1,0,2,0,4,0,0,0,0,4,5,0,5,0,1,4,0,5,2,0,1,0,6,0,2,2,0,2,2,0,0,0,1,0,6,0,1,6,2,1,4,5,5,2,1,5,5,5,6,1,2,5,6,3,1,5,4,1,3,5,5,1,1,3,3,3,5,10,3,5,6,1,3,2,3,6,6,1,1,6,1,2,3,6,2,2,5,2,2,5,4,2,1,4,1,5,4,6,5,5,1,5,2,5,1,4,4,4,4,4,1,1,1,3,5,1,6,2,1,2,3,1,2,6,4,4,3,2,2,3,2,3,1,3,5,2,3,6,2,2,6,3,4,6,5,5,1,4,4,2,3,2,2,3,6,2,4,3,5,4,5,3,3,5,5,3,5,5,4,4,3,3,4,3,4,6,3,4,6,3,4,4,4,6,3,6,4,4,4,4,4,4,4,3,4,6,3,3,3,4"
806,372,"1,5,4,0,3,3,0,1,1,0,3,2,0,0,3,5,5,0

The simulation isn't terribly fast. Depending on the system I used, each Collector's Vault "round" took between 0.5 and 1.5 seconds to calculate. I'm sure there are optimizations that could be made, or that programming in C rather than Python would improve the speed significantly. 

## basic results

For the final analysis, I ran the program over a day and calculated the outcomes of 100,000 Collector's Vaults, output in the above format to a text file that was about 80 megabytes (but, of course, has a lot of repetitive text and thus compresses well enough to be included [in the releases](https://github.com/therealchjones/collector-sim/releases)). Running the basic statistics on the file isn't terribly complicated:

In [8]:
import csv
from statistics import mean, median, stdev

file_name = "output-100000"

rounds = 0
wins = 0
turns = []
items_per_turn = [[] for i in range(len(ctp_items))]

with open(file_name, "r") as file:
    table = csv.reader(file)
    for round in table:
        items_won = [0] * len(ctp_items)
        rounds += 1
        if round[0] == "0":
            wins += 1
        turns.append(int(round[1]))
        items_this_round = list(map(int,round[2].split(',')))
        for i in items_this_round:
            items_won[i] = items_won[i] + 1
        for i in range(len(items_won)):
            items_per_turn[i].append(items_won[i])

print(f'Vaults: {rounds}')
if wins > 0:
    print(f'Wins: {wins} ({100.0 * wins / rounds}%, about 1 in {1.0 * rounds/wins})')
else:
    print('No wins.')
print(f'Mean turns (cost): {mean(turns)} ({mean(turns) * price} gold)')
print(f'Median turns (cost): {median(turns)} ({median(turns) * price} gold)')
print(f'Std. deviation of turns (cost): {stdev(turns)} ({stdev(turns) * price} gold)')
print(f'Mean items acquired:')
for i in range(len(items_per_turn)):
    print(f'{ctp_items[i][0]}: {mean(items_per_turn[i])} ({mean(items_per_turn[i])/mean(turns)}%)')

Vaults: 100000
Wins: 72 (0.072%, about 1 in 1388.888888888889)
Mean turns (cost): 380.20301 (9505075.25 gold)
Median turns (cost): 399.0 (9975000.0 gold)
Std. deviation of turns (cost): 86.12272399416882 (2153068.0998542206 gold)
Mean items acquired:
Norn Stone of each type x15: 70.13703 (0.18447257953060392%)
Rank 1 Black Anti-Matter x5: 61.53212 (0.1618401705972817%)
Norn Stone of Chaos x5: 61.5002 (0.16175621544921487%)
Gear Up Kit x25: 49.97103 (0.13143249444553318%)
Dimension Debris x30: 50.01463 (0.13154717002371968%)
3* Type Enhancement Kit x3: 59.09589 (0.15543246225220572%)
3* Enchanted Uru x1: 26.92248 (0.07081080183978554%)
Tier-2 Mega Advancement Ticket x1: 0.00191 (5.023631980188689e-06%)
Titan Component Pack x40: 0.53449 (0.0014058016005712318%)
Essence of Dimension x50: 0.30923 (0.0008133286477663604%)
Cosmic Cube Fragment x60: 0.15377 (0.0004044418270123637%)
Extreme Obelisk x1: 0.03023 (7.951015432518538e-05%)


The basics also aren't terribly surprising:
- You have the same chance to "win" as everyone else, so the chances you'll get the big prize is about 1 in 1,500, the same as getting chosen randomly from the list of players
- The number of turns (or purchases) it takes to win is pretty variable, but centers on just under 400, or about 10 million gold. Again, assuming you all start at the same time and go the same speed, this is what you and all the other players spend even if you don't win. (This average required number of turns can be calculated from the probabilities of getting the 4 "memento" items, but overestimates the result. See [Why we simulate](#why-we-simulate).)
- The amount of stuff you get is modest. More details (including a more readable breakdown of quantities) are available in [Choosing a Vault](#choosing-a-vault).

## making decisions

Knowledge is great, and certainly this is pretty good evidence that it may not be worth playing the Collector's Vault if you're not willing to spend about 10 million gold. But is there any more useful advice about how to play? 

### playing against the assumptions

First, let's check our [assumptions](#the-assumptions)
1. Though we've focused on the case where the vault is full, the chances of winning is about the same as the chances of being randomly selected from all the users who join. Therefore, if you join a vault that has (and is expected to continue to have) fewer users, you have a higher chance of winning.
2. Similarly, if most users are going to join late (is everybody playing Alliance Conquest or World Event?), you may have the advantage of getting more turns to yourself early on. Along the same lines (though probably more difficult to guess) if you've got a particularly good connection to the Netmarble servers you may be able to take turns more quickly than others. All these will result in more turns (and thus more spending) for you, with some minor increase in the chances of winning. 

We can check all of these possibilities by just giving some players fewer turns. Let's take our same 1500 players (remember, you're number 0) and suppose, say, half of them don't start until the others have already taken 100 turns:

In [10]:
# Warning: change or re-run this cell if you wish, but as written it'll take
# around 10-20 minutes to duplicate the currently saved output

def do_underfilled_round():
    winner = -1
    turns = -1
    users_items = [[] for i in range(users)]
    round_items = ctp_items.copy()
    for turn in range(3600*100):
        random.shuffle(user_nums)
        for user_num in user_nums:
            if user_num % 2 == 0 or turn > 99:
                if add_user_item( user_num, get_item(round_items), users_items):
                    turns = len(users_items[0])
                    winner = user_num
                    break
        if turns >= 0:
            break
    return (winner,turns,users_items[0])

underfilled_sims = 1000

underfilled_rounds = 0
underfilled_wins = 0
underfilled_turns = []
underfilled_items = [[] for i in range(len(ctp_items))]
for i in range(underfilled_sims):
    round = do_underfilled_round()
    items_won = [0] * len(ctp_items)
    underfilled_rounds += 1
    if round[0] == "0":
        underfilled_wins += 1
    underfilled_turns.append(round[1])
    for i in round[2]:
        items_won[i] = items_won[i] + 1
    for i in range(len(items_won)):
        underfilled_items[i].append(items_won[i])

print(f'Vaults: {underfilled_rounds}')
if underfilled_wins > 0:
    print(f'Wins: {underfilled_wins} ({100.0 * underfilled_wins / underfilled_rounds}%, about 1 in {1.0 * underfilled_rounds/underfilled_wins})')
else:
    print('No wins.')
print(f'Mean turns (cost): {mean(underfilled_turns)} ({mean(underfilled_turns) * price} gold)')
print(f'Median turns (cost): {median(underfilled_turns)} ({median(underfilled_turns) * price} gold)')
print(f'Std. deviation of turns (cost): {stdev(underfilled_turns)} ({stdev(underfilled_turns) * price} gold)')
print(f'Mean items acquired:')
for i in range(len(underfilled_items)):
    print(f'{ctp_items[i][0]}: {mean(underfilled_items[i])} ({mean(underfilled_items[i])/mean(underfilled_turns)}%)')

Vaults: 1000
No wins.
Mean turns (cost): 424.126 (10603150.0 gold)
Median turns (cost): 444.5 (11112500.0 gold)
Std. deviation of turns (cost): 93.94554747240494 (2348638.6868101233 gold)
Mean items acquired:
Norn Stone of each type x15: 81.07 (0.19114602735979402%)
Rank 1 Black Anti-Matter x5: 68.337 (0.16112428853689706%)
Norn Stone of Chaos x5: 68.61 (0.16176796518015873%)
Gear Up Kit x25: 55.181 (0.1301052045854298%)
Dimension Debris x30: 55.027 (0.12974210494051297%)
3* Type Enhancement Kit x3: 65.015 (0.15329171048226237%)
3* Enchanted Uru x1: 29.704 (0.07003579125071324%)
Tier-2 Mega Advancement Ticket x1: 0.002 (4.7155798041148155e-06%)
Titan Component Pack x40: 0.631 (0.001487765428198224%)
Essence of Dimension x50: 0.344 (0.0008110797263077481%)
Cosmic Cube Fragment x60: 0.164 (0.00038667754393741484%)
Extreme Obelisk x1: 0.041 (9.666938598435371e-05%)


Sure enough, more turns, more gold, and the chances of winning are small enough that it's probably not an appreciable difference. (If it were 1/750 instead of 1/1500, would it make a practical difference to you?)

### choosing a vault

If it's very unlikely we'll win the "big prize", is it worth playing at all? Some will no doubt argue "you can't win if you don't play". Objectively, let's see what you get for your money:

As noted in the [basic results](#basic-results), we've got the items we'll acquire (on average) from a single Vault. To make the quantities more understandable, though, we'll have to take the number of each item times the number of times we get that item:

In [11]:
def get_item_quantity( item_index, items_acquired):
    item_description = str(ctp_items[item_index][0])
    [item_name, item_number] = item_description.split(" x")
    item_number = int(item_number)
    return ( item_name, item_number * items_acquired)

def list_acquired_items(acquired_items):
    for i in range(len(acquired_items)):
        (name, number) = get_item_quantity( i, mean(acquired_items[i]))
        print( f'{name}: {str(number)}')

list_acquired_items(items_per_turn)

Norn Stone of each type: 1052.0554499999998
Rank 1 Black Anti-Matter: 307.6606
Norn Stone of Chaos: 307.501
Gear Up Kit: 1249.27575
Dimension Debris: 1500.4388999999999
3* Type Enhancement Kit: 177.28767
3* Enchanted Uru: 26.92248
Tier-2 Mega Advancement Ticket: 0.00191
Titan Component Pack: 21.3796
Essence of Dimension: 15.461500000000001
Cosmic Cube Fragment: 9.226199999999999
Extreme Obelisk: 0.03023


So for your 10 million gold, about a thousand random Norn, a few hundred Chaos Norn & Black Anti-Matter, and a handful of T2/T3 material. Not great, but honestly a little better than I expected.

What if instead we decided to use the other Vault?

In [12]:
def get_artifact_item_quantity( item_index, items_acquired):
    item_description = str(artifact_items[item_index][0])
    [item_name, item_number] = item_description.split(" x")
    item_number = int(item_number)
    return ( item_name, item_number * items_acquired)

def list_artifact_acquired_items(acquired_items):
    for i in range(len(acquired_items)):
        (name, number) = get_artifact_item_quantity( i, mean(acquired_items[i]))
        print( f'{name}: {str(number)}')

list_artifact_acquired_items(items_per_turn)

Gear Up Kit: 1402.7405999999999
Dimension Debris: 1230.6424
Norn Stone of each type: 922.503
Rank 1 Black Anti-Matter: 249.85514999999998
Norn Stone of Chaos: 250.07315
3* ISO-8: 59.09589
Lv. 2 Artifact: 26.92248
Mega Uniform Upgrade Ticket: Mythic: 0.00191
Essence of Dimension: 21.3796
Cosmic Cube Fragment: 15.461500000000001
Titan Component Pack: 9.226199999999999
6* Rank Up Ticket: 0.03023


A little lighter on BAM & Chaos Norn, ISO-8 & Artifact fodder instead of TEK & Uru, and the T2/T3 material quantities are small enough the differences probably don't matter. (If you need a couple of hundred extra GUK for your 10 million, by all means choose this one.)

Finally, of course, the big question whose result is a surprise to, well, nobody:

<a id="should-you-play-collectors-vault"></a>
## should you play collector's vault?

It's entirely up to you. There's a 1 in 1,500 chance you'll win the big prize for 10 million gold. The materials you get aren't usually available for gold in the shop, and certainly spending the gold here is better than spending crystals in the shop, but none of the ones you're likely to get are at all hard to come by during regular game play.

If you are going to play it, start as soon as the vault opens, use the "Auto Purchase" function and buy stuff as quickly as you can, and choose the vault based on whether you'd prefer to get low-level Uru & TEK or low-level Artifacts & ISO.

All of which you could probably have figured out without a simulation.

## why we simulate

As noted in the [basic results](#basic-results), it's possible to calculate (or at least estimate) many of the results with basic statistics rather than using a simulation. For instance, we can estimate how many turns are needed to "finish" a Vault by calculating the probability of some user having all 4 "memento" items after *n* turns:

In [13]:
p_1 = 0.001248
p_2 = 0.000720
p_3 = 0.000360
p_4 = 0.000072

# probability of getting a specific item within n turns
# is 1 - (probability of not getting that item in n turns)
def p_n( p, n ):
    return 1 - (1-p)**n

# probability of getting item a and item b within n turns
# is p_n(a) * p_n(b)
def p_all(n):
    return p_n(p_1, n) * p_n(p_2, n) * p_n(p_3,n) * p_n(p_4, n)

# the round ends when any of the users gets all 4, so the average number
# of turns it takes is when the chance of having all 4 is about
# 1 / number of users
for i in range(1000):
    if p_all(i) > 1.0/users:
        print (f'Average number of turns: {i}')
        break

Average number of turns: 472


However, we can see that this appears to overestimate the number of turns required (around 400); that's because it doesn't take into account other (non-memento) items running out (and thus increasing the probability of getting the memento items) as the number of turns increases. In addition, this doesn't calculate what you are likely to acquire, just what you're likely to spend. These are two of the benefits of using the simulation.

## see also

Simulating stuff for an inconsequential game or otherwise using data for frivolous decision making isn't for everyone. If it interests you, though, consider checking out the stuff in the [datamine section of the MFF subreddit](https://www.reddit.com/r/future_fight/?f=flair_name%3A%22Leak%20%2F%20Datamine%22) and other work by [u/OwO_PinkChode_OwO](https://www.reddit.com/user/OwO_PinkChode_OwO/) (including simulations of costs for advancement and much more) at https://21000dollor.com.

## development

This project is available [on GitHub](https://github.com/therealchjones/collector-sim) (though this notebook is best viewed <a id="nbviewer-link" href="https://nbviewer.org/github/therealchjones/collector-sim/blob/master/Collector%27s%20Vault.ipynb">on nbviewer.org</a>)). Instructions for downloading the associated files, including all source code, are in the [README](https://github.com/therealchjones/collector-sim/blob/master/README.md). 

<script>
  var urlBits = window.location.href.split('/');
  var revision = null;
  for ( let i = 0; i < urlBits.length; i++ ) {
    if ( urlBits[i] == "blob" && urlBits.length > i+1 ) revision = urlBits[i+1];
  }
  if ( revision ) {
    document.getElementById("nbviewer-link").setAttribute(
      "href",
      "https://nbviewer.org/github/therealchjones/collector-sim/blob/"
      + revision
      + "Collecto%27s%20Vault.ipynb"
    );
  }
</script>

## copyright

  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <a rel="dct:publisher"
     href="https://github.com/therealchjones">
    <span property="dct:title">Christian Jones</span></a>
  has waived all copyright and related or neighboring rights to
  <span property="dct:title">Simulating the Collector's Vault</span>.
This work is published from:
<span property="vcard:Country" datatype="dct:ISO3166"
      content="US" about="https://github.com/therealchjones">
  United States</span>.